# Use public data to replicate C.Sippl (2016) *Technophysics* results Figure 3

## Imports, global setup

In [ ]:
import os

import numpy as np
import pandas as pd

import rf
import rf.imaging

import matplotlib.pyplot as plt
import seaborn as sns

from obspy import UTCDateTime

In [ ]:
import seismic.receiver_fn.rf_util as rf_util
import seismic.receiver_fn.rf_plot_utils as rf_plot_utils
import seismic.receiver_fn.rf_stacking as rf_stacking

In [ ]:
# import importlib
# importlib.reload(rf_stacking)

## Load the data file of processed RF traces for analysis

In [ ]:
network = '7W'
# network = '6F'
# rf_type = 'ZRT_fd'
# rf_type = 'ZRT_td'
rf_type = 'ZRT_it'
data = rf_util.read_h5_rf(r"..\DATA\7W_rfs_20080827T000136-20101231T235620_{}_qual.h5".format(rf_type))
# data = rf_util.read_h5_rf(r"..\DATA\6F_rfs_20080827T032712-20110524T003459_{}_qual.h5".format(rf_type))
data

In [ ]:
stations = set([tr.stats.station for tr in data])
station_idx = {st: data.select(station=st) for st in stations}
len(station_idx)

In [ ]:
stations_not_empty = set([st for st in station_idx if len(station_idx[st]) > 0])
len(stations_not_empty)

In [ ]:
test_stations = sorted(stations_not_empty)
print(test_stations)

In [ ]:
primary_station = 'BL05'
# primary_station = 'BL20'

## Process data and present RF stacking

### Filter data down to selected station

In [ ]:
data_rf = data.select(station=primary_station)
len(data_rf)

In [ ]:
# Preview first 100 RF plots
# _ = rf_plot_utils.plot_rf_stack(data_rf.sort(['back_azimuth'])[0:100], time_window=(-5,30))

In [ ]:
event_id_all = np.array(sorted(list(set([tr.stats.event_id for tr in data_rf]))))

In [ ]:
np.savetxt("7X.BL05_event_ids_all.txt", event_id_all, fmt='%s')

### Check the main data channel code and set channel accordingly

In [ ]:
set([tr.stats.channel for tr in data_rf])

In [ ]:
channel = 'BHR'

In [ ]:
data_rf = data_rf.select(channel=channel)
len(data_rf)

## Replicate as much of Sippl's trace filtering logic as possible

### Filter RFs with too large amplitude or R-component maximum not near onset

We use library function with lax thresholds on SNR and RMS amplitude, so that filtering is determined by max. amplitude and location of highest peak.

In [ ]:
rf_util.label_rf_quality_simple_amplitude(rf_type, data_rf, snr_cutoff=1.0, rms_amp_cutoff=1.0, max_amp_cutoff=2.0)
data_good = rf.RFStream([tr for tr in data_rf if tr.stats.predicted_quality == 'a'])

In [ ]:
len(data_good)

In [ ]:
_ = rf_plot_utils.plot_rf_stack(data_good.sort(['back_azimuth'])[0:100], time_window=(-5,30))

### Filter by cross-correlation coefficient against other traces

In [ ]:
data_good = rf_util.filter_crosscorr_coeff(data_good, time_window=(-2, 20)).sort(['back_azimuth'])
len(data_good)

In [ ]:
event_id_good = np.array(sorted(list(set([tr.stats.event_id for tr in data_good]))))

In [ ]:
np.savetxt("7X.BL05_event_ids_filt.txt", event_id_good, fmt='%s')

In [ ]:
_ = rf_plot_utils.plot_rf_stack(data_good.sort(['back_azimuth'])[0:100], time_window=(-5,30))

In [ ]:
_ = rf_plot_utils.plot_station_rf_overlays({channel: data_good}, time_range=(-5, 30))

### Plot the good RFs

In [ ]:
time_window=(-5.0, 30.0)

In [ ]:
save_file = 'RF_stack_{}.{}.{}_{}_validation.png'.format(network, primary_station, channel, rf_type)
fig = rf_plot_utils.plot_rf_stack(data_good, save_file=save_file, dpi=300, time_window=time_window)
plt.show()

## H-k stacking

In [ ]:
db = rf_util.rf_to_dict(data_good)
data_sta = db[primary_station]

In [ ]:
# weighting = (0.5, 0.5, 0.0)
weighting = (0.33, 0.33, 0.33)

V_p = 6.4
k_grid, h_grid, hk_stack = rf_stacking.compute_hk_stack(data_sta, channel, V_p=V_p, h_range=np.linspace(20.0, 60.0, 201),
                                                        k_range=np.linspace(1.5, 2.0, 251), root_order=2)
# Inferred V_p:
# k_grid, h_grid, hk_stack = rf_stacking.compute_hk_stack(data_sta, channel, h_range=np.linspace(20.0, 60.0, 201),
#                                                         k_range=np.linspace(1.5, 2.0, 251), root_order=2)

# Sum the phases
hk_stack_sum = rf_stacking.compute_weighted_stack(hk_stack, weighting)

# Raise the final sum over phases to power >1 to increase contrast
hk_stack_sum = rf_util.signed_nth_power(hk_stack_sum, 2)
hk_stack_sum = hk_stack_sum/np.nanmax(hk_stack_sum[:])

In [ ]:
# Numerically find location of maxima
best_solution = rf_stacking.find_local_hk_maxima(k_grid, h_grid, hk_stack_sum)[0]
print("Best solution (H, k, stack, row, col) = {}".format(best_solution))
h_max, k_max = best_solution[0:2]

sta = data_sta[channel][0].stats.station
num = len(data_sta[channel])
save_file = 'Hk_stack_{}.{}.{}_{}_validation.png'.format(network, sta, channel, rf_type)
fig = rf_plot_utils.plot_hk_stack(k_grid, h_grid, hk_stack_sum, title='Station ' + network + '.' + sta + '.' + channel, num=num)
plt.scatter(k_max, h_max, marker='x', c="#40ff40", s=100, alpha=0.8, zorder=100)
if k_max >= 1.75:
    plt.text(k_max - 0.01, h_max + 1, "Solution H = {:.3f}, k = {:.3f}".format(h_max, k_max),
             color="#40ff40", fontsize=16, horizontalalignment='right', zorder=100)
else:
    plt.text(k_max + 0.01, h_max + 1, "Solution H = {:.3f}, k = {:.3f}".format(h_max, k_max),
             color="#40ff40", fontsize=16, zorder=100)
# end if
# fig.savefig(save_file, dpi=300)
plt.show()

## Extend validation to cover other stations of Bilby deployment

In [ ]:
db_7W = rf_util.rf_to_dict(data)

In [ ]:
output_folder = 'csippl_validation_it_rev2_7W'
# output_folder = 'csippl_validation_td_rev1_7W'
# output_folder = 'csippl_validation_td_rev1_6F'
# output_folder = 'csippl_validation_fd_rev1_7W'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [ ]:
for test_station in test_stations:
    try:
        db_station = db_7W[test_station]
        channel = rf_util.choose_rf_source_channel(rf_type, db_station)

        db_channel = db_station[channel]
        test_rf = rf.RFStream(db_channel)

        rf_util.label_rf_quality_simple_amplitude(rf_type, test_rf, snr_cutoff=1.0, rms_amp_cutoff=1.0, max_amp_cutoff=2.0)
        data_good = rf.RFStream([tr for tr in test_rf if tr.stats.predicted_quality == 'a'])
        data_good = rf_util.filter_crosscorr_coeff(data_good, time_window=(-2, 20)).sort(['back_azimuth'])
        print("Num traces = {}".format(len(data_good)))

        save_file = 'RF_stack_{}.{}.{}_{}_validation.png'.format(network, test_station, channel, rf_type)
        save_file = os.path.join(output_folder, save_file)
        fig = rf_plot_utils.plot_rf_stack(data_good, save_file=save_file, dpi=300, time_window=time_window)

        db_good = rf_util.rf_to_dict(data_good)
        data_sta = db_good[test_station]

#         weighting = (0.5, 0.5, 0.0)
        weighting = (0.33, 0.33, 0.33)

        V_p = 6.4
        k_grid, h_grid, hk_stack = rf_stacking.compute_hk_stack(data_sta, channel, V_p=V_p, h_range=np.linspace(20.0, 60.0, 201),
                                                                k_range=np.linspace(1.5, 2.0, 251), root_order=2)

        # Sum the phases
        hk_stack_sum = rf_stacking.compute_weighted_stack(hk_stack, weighting)

        # Raise the final sum over phases to power >1 to increase contrast
        hk_stack_sum = rf_util.signed_nth_power(hk_stack_sum, 2)
        hk_stack_sum = hk_stack_sum/np.nanmax(hk_stack_sum[:])

        # Numerically find location of maxima
        best_solution = rf_stacking.find_local_hk_maxima(k_grid, h_grid, hk_stack_sum)[0]
        print("Best solution (H, k, stack, row, col) = {}".format(best_solution))
        h_max, k_max = best_solution[0:2]

        sta = test_station
        num = len(data_sta[channel])
        save_file = 'Hk_stack_{}.{}.{}_{}_validation.png'.format(network, sta, channel, rf_type)
        save_file = os.path.join(output_folder, save_file)
        fig = rf_plot_utils.plot_hk_stack(k_grid, h_grid, hk_stack_sum, title='Station ' + network + '.' + sta + '.' + channel, num=num)
        plt.scatter(k_max, h_max, marker='x', c="#40ff40", s=100, zorder=100)
        if k_max >= 1.75:
            plt.text(k_max - 0.01, h_max + 1, "Solution H = {:.3f}, k = {:.3f}".format(h_max, k_max),
                     color="#40ff40", fontsize=16, horizontalalignment='right', zorder=100)
        else:
            plt.text(k_max + 0.01, h_max + 1, "Solution H = {:.3f}, k = {:.3f}".format(h_max, k_max),
                     color="#40ff40", fontsize=16, zorder=100)
        # end if
        fig.savefig(save_file, dpi=300)
        plt.show()

    except Exception as e:
        print("Failed on station {} with error:\n{}".format(test_station, str(e)))
    # end try
# end for

--------------